In [ ]:
%pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
reactome = pd.read_csv('/content/FIsInGene_04142025_with_annotations.txt', sep='\t',usecols=[0, 1 ,4])  #contém a rede de interação gênica do Reactome - a função lê só as colunas de índice 0, 1 E 4 - tabela/dataframe do pandas, não a rede em si


In [ ]:
G_reactome = nx.from_pandas_edgelist(reactome, 'Gene1', 'Gene2', 'Score')
print(f"Number of edges in reactome dataset: {G_reactome.number_of_edges()}")
print(f"Number of nodes in reactome dataset: {G_reactome.number_of_nodes()}")

Number of edges in reactome dataset: 272622
Number of nodes in reactome dataset: 13733


# Separando por Score

In [ ]:
df_gene_class = pd.read_csv('/content/Gene_Classification.tsv', sep='\t', usecols=[0, 1]) #contém a lista de classificação dos genes em "TSG" ou "Oncongene"

In [ ]:
df_TSG = df_gene_class[df_gene_class['ROLE_IN_CANCER'].str.contains('TSG', case=False, na=False)]
df_TSG.loc[:, 'GENE_SYMBOL'].to_csv('genes_TSG.txt', index=False)

In [ ]:
df_oncogene = df_gene_class[df_gene_class['ROLE_IN_CANCER'].str.contains('oncogene', case=False, na=False)]
df_oncogene.loc[:, 'GENE_SYMBOL'].to_csv('genes_oncogene.txt', index=False)

In [ ]:
# Filtra os genes da lista maior que estão na lista de TSG
df_antes_score = reactome[reactome['Gene'].isin(df_TSG['Gene'])]

In [ ]:
# Separar por Score

print("Antes de filtrar:")
print(f"Number of nodes: {G_reactome.number_of_nodes()}")
print(f"Number of edges: {G_reactome.number_of_edges()}")
print()

reactome_filtrado = reactome[reactome['Score']>0.9]


# Cria um grafo a partir do dataframe
G_reactome_filtrado = nx.from_pandas_edgelist(reactome_filtrado, 'Gene1', 'Gene2',edge_attr='Score')

print("Depois de filtrar:")
print(f"Number of nodes: {G_reactome_filtrado.number_of_nodes()}")
print(f"Number of edges: {G_reactome_filtrado.number_of_edges()}")

#100%

# Antes de filtrar:
# Number of nodes: 13733
# Number of edges: 272622

# Depois de filtrar:
# Number of nodes: 10022
# Number of edges: 194494

#Nós removidos: ~27,02%
#Arestas removidas: ~28,66%


#95%

# Antes de filtrar:
# Number of nodes: 13733
# Number of edges: 272622

# Depois de filtrar:
# Number of nodes: 10980
# Number of edges: 202231

#Remoção de nós: ~20,04%
#Remoção de arestas: ~25,83%


#90%

# Antes de filtrar:
# Number of nodes: 13733
# Number of edges: 272622

# Depois de filtrar:
# Number of nodes: 13046
# Number of edges: 231249

#Remoção de nós: ~5,00%
#Remoção de arestas: ~15,17%

Antes de filtrar:
Number of nodes: 13733
Number of edges: 272622

Depois de filtrar:
Number of nodes: 13046
Number of edges: 231249


# Separando o componente principal

In [ ]:
#Extrair componente principal
if not nx.is_connected(G_reactome_filtrado): #verifica se todo o grafo está conectado - se todos os nós conseguem se alcançar entre si por algum caminho.
  componentes = nx.connected_components(G_reactome_filtrado)
  maior = max(componentes, key=len)
  G_principal = G_reactome_filtrado.subgraph(maior).copy()

print(G_principal.number_of_nodes())
print(G_principal.number_of_edges())

12425
230760


# Extraindo as medidas de centralidade

In [ ]:
node_degrees = dict(G_principal.degree())

# converte o dicionário para um dataframe do pandas
degree_df = pd.DataFrame(node_degrees.items(), columns=['Gene', 'Grau'])

In [ ]:
# salva o DataFrame em um arquivo CSV
degree_df.to_csv('node_degrees.csv', index=False)

In [ ]:
node_closeness = dict(nx.closeness_centrality(G_principal))

clos_df = pd.DataFrame(node_closeness.items(), columns=['Gene', 'Closeness'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1617957241.py", line 1, in <cell line: 0>
    node_closeness = dict(nx.closeness_centrality(G_principal))
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/networkx/utils/decorators.py", line 784, in func
    return argmap._lazy_compile(__wrapper)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<class 'networkx.utils.decorators.argmap'> compilation 26", line 3, in argmap_closeness_centrality_23
    import gzip
            ^^^^
  File "/usr/local/lib/python3.11/dist-packages/networkx/utils/backends.py", line 929, in _call_if_any_backends_installed
    return self._call_with_backend(backend_name, args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [ ]:
clos_df.to_csv('node_closeness.csv', index=False)

In [ ]:
node_betweenness = dict(nx.betweenness_centrality(G_principal))

betw_df = pd.DataFrame(node_betweenness.items(), columns=['Gene', 'Betweenness'])

print (len(clos_df))

In [ ]:
betw_df.to_csv('node_betweenness.csv', index=False)

In [ ]:
node_eigenvector = dict(nx.eigenvector_centrality(G_principal))

eigen_df = pd.DataFrame(node_eigenvector.items(), columns=['Gene', 'Eigenvector'])

print (len(clos_df))

In [ ]:
eigen_df.to_csv('node_eigenvector.csv', index=False)

# Separando por Oncogene e TSG

In [ ]:
df_gene_class = pd.read_csv('/content/Gene_Classification.tsv', sep='\t', usecols=[0, 1])

In [ ]:
df_TSG = df_gene_class[df_gene_class['ROLE_IN_CANCER'].str.contains('TSG', case=False, na=False)]
df_TSG.loc[:, 'GENE_SYMBOL'].to_csv('genes_TSG.txt', index=False)

In [ ]:
df_oncogene = df_gene_class[df_gene_class['ROLE_IN_CANCER'].str.contains('oncogene', case=False, na=False)]
df_oncogene.loc[:, 'GENE_SYMBOL'].to_csv('genes_oncogene.txt', index=False)

In [ ]:
df_TSG = pd.read_csv("/content/genes_TSG.txt")
df_oncogene = pd.read_csv("/content/genes_oncogene.txt")

In [ ]:
# Remove genes que estão nas duas listas
genes_comuns = set(df_TSG["GENE_SYMBOL"]) & set(df_oncogene["GENE_SYMBOL"])

# Filtra para manter apenas os genes exclusivos
df_TSG = df_TSG[~df_TSG["GENE_SYMBOL"].isin(genes_comuns)]
df_oncogene = df_oncogene[~df_oncogene["GENE_SYMBOL"].isin(genes_comuns)]

df_TSG.to_csv('genes_TSG.txt', index=False)
df_oncogene.to_csv('genes_oncogene.txt', index=False)

In [ ]:
print(len(df_TSG))
print(len(df_oncogene))

252
253


# Separando as Med. Cen. dos TSGs e Oncogenes

In [ ]:
# Carrega a lista de genes TSG
df_TSG = pd.read_csv("/content/genes_TSG.txt", header=None, names=["Gene"], skiprows=1)

In [ ]:
# Carrega a lista de genes Oncogene
df_oncogene = pd.read_csv("/content/genes_oncogene.txt", header=None, names=["Gene"], skiprows=1)

*   **Betweenness**

In [ ]:
# Carrega a lista maior de genes (ex: todos os genes e seu Betweenness)
df_maior_betw = pd.read_csv("/content/drive/MyDrive/Projeto - Redes Complexas/Artigo/MC_genes_geral/Betweenness/Componente principal/node_betweenness.csv", sep=',', usecols=[0,1])

In [ ]:
# Filtra os genes da lista maior que estão na lista de TSG
df_TSG_betw = df_maior_betw[df_maior_betw['Gene'].isin(df_TSG['Gene'])]

# Salva o resultado em um novo arquivo
df_TSG_betw.to_csv("TSG_betw.csv", index=False)

In [ ]:
print(df_TSG_betw['Betweenness'].max())

0.0758569375657087


In [ ]:

# Filtra os genes da lista maior que estão na lista de Oncogene
df_onc_betw = df_maior_betw[df_maior_betw['Gene'].isin(df_oncogene['Gene'])]

# Salva o resultado em um novo arquivo
df_onc_betw.to_csv("onc_betw.csv", index=False)

In [ ]:
print(len(df_onc_betw))

242


*   **Closeness**

In [ ]:
# Carrega a lista maior de genes (ex: todos os genes e seus Betweenness)
df_maior_clos = pd.read_csv("/content/drive/MyDrive/Projeto - Redes Complexas/Artigo/MC_genes_geral/Closeness/Componente principal/node_closeness.csv", sep=',', usecols=[0,1])

In [ ]:
# Filtra os genes da lista maior que estão na lista de TSG
df_TSG_clos = df_maior_clos[df_maior_clos['Gene'].isin(df_TSG['Gene'])]

# Salva o resultado em um novo arquivo
df_TSG_clos.to_csv("TSG_clos.csv", index=False)

In [ ]:
print(df_TSG_clos['Closeness'].max())

0.4278384241881607


In [ ]:
# Filtra os genes da lista maior que estão na lista de Oncogene
df_onc_clos = df_maior_clos[df_maior_clos['Gene'].isin(df_oncogene['Gene'])]

# Salva o resultado em um novo arquivo
df_onc_clos.to_csv("onc_clos.csv", index=False)

In [ ]:
print(df_TSG_clos['Closeness'].min())

0.1877049056489749


*   **Grau**

In [ ]:
# Carrega a lista maior de genes (ex: todos os genes e seu Betweenness)
df_maior_degree = pd.read_csv("/content/drive/MyDrive/Projeto - Redes Complexas/Artigo/MC_genes_geral/Grau/Componente principal/node_degrees.csv", sep=',', usecols=[0,1])

In [ ]:
# Filtra os genes da lista maior que estão na lista de TSG
df_TSG_degree = df_maior_degree[df_maior_degree['Gene'].isin(df_TSG['Gene'])]

# Salva o resultado em um novo arquivo
df_TSG_degree.to_csv("TSG_degree.csv", index=False)

In [ ]:
print(df_TSG_degree['Grau'].min())

1


In [ ]:
# Filtra os genes da lista maior que estão na lista de Oncogene
df_onc_degree = df_maior_degree[df_maior_degree['Gene'].isin(df_oncogene['Gene'])]

# Salva o resultado em um novo arquivo
df_onc_degree.to_csv("onc_degree.csv", index=False)

In [ ]:
print(len(df_onc_degree))

242


*   **Eigenvector**

In [ ]:
# Carrega a lista maior de genes (ex: todos os genes e seu Betweenness)
df_maior_eigen = pd.read_csv("/content/drive/MyDrive/Projeto - Redes Complexas/Artigo/MC_genes_geral/Eigenvector/Componente principal/node_eigenvector.csv", sep=',', usecols=[0,1])

In [ ]:
# Filtra os genes da lista maior que estão na lista de TSG
df_TSG_eigen = df_maior_eigen[df_maior_eigen['Gene'].isin(df_TSG['Gene'])]

# Salva o resultado em um novo arquivo
df_TSG_eigen.to_csv("TSG_eigen.csv", index=False)

In [ ]:
print(df_TSG_eigen['Eigenvector'].min())

2.5800082540280804e-09


In [ ]:
# Filtra os genes da lista maior que estão na lista de Oncogene
df_onc_eigen = df_maior_eigen[df_maior_eigen['Gene'].isin(df_oncogene['Gene'])]

# Salva o resultado em um novo arquivo
df_onc_eigen.to_csv("onc_eigen.csv", index=False)

In [ ]:
print(len(df_onc_eigen))

242
